# Jane Street January 2025 Puzzle

Link to current puzzle: https://www.janestreet.com/puzzles/current-puzzle/

Tentative archive link (currently not archived): https://www.janestreet.com/puzzles/current-puzzle/somewhat-square-sudoku-index/


In [1]:
from Sudoku import Sudoku

NULL_INT = 1

puzzle_st = (".......2."
             "........5"
             ".2......."
             "..0......"
             "........."
             "...2....."
             "....0...."
             ".....2..."
             "......5..")
puzzle_lst = []
for i in range(0, 81, 9):
    puzzle_lst.append([])
    for ch in puzzle_st[i:i+9]:
        if ch != '.':
            puzzle_lst[-1].append(int(ch))
        else:
            puzzle_lst[-1].append(NULL_INT)

puzzle_lst

[[1, 1, 1, 1, 1, 1, 1, 2, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 5],
 [1, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 2, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 5, 1, 1]]

In [2]:
game = Sudoku(puzzle_lst, is_hyper_Sudoku=True) #null_digit = -1, valid_digits = set(range(10)))
print(game)

[1, 1, 1, 1, 1, 1, 1, 2, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 5]
[1, 2, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 2, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 2, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 5, 1, 1]



In [3]:
import test_Sudoku

runner = test_Sudoku.unittest.TextTestRunner(verbosity=2)
runner.run(test_Sudoku.suite())

check_unique_test (test_Sudoku.SudokuTest) ... ok
check_row_test (test_Sudoku.SudokuTest) ... ok
check_col_test (test_Sudoku.SudokuTest) ... ok
check_box_test (test_Sudoku.SudokuTest) ... ok
find_options_test (test_Sudoku.SudokuTest) ... ok
candidates_test (test_Sudoku.SudokuTest) ... ok
candidates_advanced (test_Sudoku.SudokuTest) ... ok
impossible_test (test_Sudoku.SudokuTest) ... ok
impossible_x_test (test_Sudoku.SudokuTest) ... ok
check_done (test_Sudoku.SudokuTest) ... ok
check_null_digit (test_Sudoku.SudokuTest) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.016s

OK


<unittest.runner.TextTestResult run=11 errors=0 failures=0>

In [4]:
import test_solver

runner = test_solver.unittest.TextTestRunner(verbosity=2)
runner.run(test_solver.suite())

check_solver (test_solver.SudokuSolverTest) ... ok
check_solved_puzzles (test_solver.SudokuSolverTest) ... ok
solve_X_sudoku (test_solver.SudokuSolverTest) ... ok
solve_hyper_sudoku (test_solver.SudokuSolverTest) ... ok
solve_hyper_sudoku_x (test_solver.SudokuSolverTest) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.230s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [5]:
from copy import deepcopy
from typing import List
import random
import time

from Sudoku import Sudoku
from str2grid import str2grid, grid2str
def solve(game0: Sudoku, progress_factor=1.0):
    global calls, depth_max, progress, progress_update, update_increment
    stacc = [(game0, 0)]
    while stacc:
        if verbose and progress > progress_update:
            print("%.1f" % (progress*100), end=' ...')
            progress_update = (
                (progress//update_increment) + 1) * update_increment
            
        game, depth = stacc.pop()
        calls += 1
        depth_max = max(depth, depth_max)
        # solved = False
        # while not solved:
        solved = True  # assume solved
        edited = False  # if no edits, either done or stuck
        deadend = False
        for i in range(game.n):
            if deadend: break
            for j in range(game.n):
                if game.grid[i][j] == game._null_digit:
                    solved = False
                    options = game.candidates[i][j]
                    if len(options) == 0:
                        progress += progress_factor
                        deadend = True #return False  # this call is going nowhere
                        break
                    elif len(options) == 1:  # Step 1
                        game.place_and_erase(
                            i, j, list(options)[0])  # Step 2
                        edited = True
                        deadend = True
                        break
        if edited:
            stacc.append((game, depth))
            continue

        if solved:
            progress += progress_factor
            if not all_solutions:
                return grid2str(game.grid.copy())
            yield grid2str(game.grid.copy())

        # backtracking check point:
        nxt_guess, min_options = None, 10
        for i in range(9):
            for j in range(9):
                if len(game.candidates[i][j]) < min_options and len(game.candidates[i][j]) > 1:
                    nxt_guess, min_options = (i, j), len(game.candidates[i][j])
        if not nxt_guess:
            continue
        i, j = nxt_guess
        options = game.candidates[i][j]
        progress_factor *= (1/len(options))
        for y in options:
            game_next = deepcopy(game)
            game_next.place_and_erase(i, j, y)
            # game_next.flush_candidates() # full grid cleaning
            stacc.append((game_next, depth+1))


calls, depth_max = 0, 0
progress, update_increment, progress_update = 0.0, 0.01, 0.01
solution_set = []
verbose = True
all_solutions=True

: 

In [ ]:
# from solver import solve_sudoku
from str2grid import str2grid, grid2str
import re

UNUSED = set(range(0, 10))
UNUSED -= {2, 5, 0}
puzzle_st_template = (  ".......2."
                        "........5"
                        ".2......."
                        "..0......"
                        "........."
                        "...2....."
                        "....0...."
                        ".....2..."
                        "......5..")

for n in {1}:
    attempt = re.sub(r'\.', str(n), puzzle_st_template)
    grid = str2grid(attempt)
    cgame = Sudoku(grid, is_hyper_Sudoku=True, null_digit=n, valid_digits= set(range(10))-{n})
    print('Now finding solutions for\n', grid2str(grid))

    sols = solve(cgame)
    i = 1
    with open('solutions.txt', 'w') as f:
        for s in sols:
            f.write(s + '\n')
            if i % 10 == 1:
                print('\n'.join(str(i) for i in (str2grid(s))))
                print('='*12)
            i += 1